<a href="https://colab.research.google.com/github/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/blob/main/05_transfer_learning_fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lets start with importing some helper functions from a script

In [1]:
!wget  https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py

--2025-09-17 15:57:37--  https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6733 (6.6K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   6.58K  --.-KB/s    in 0s      

2025-09-17 15:57:38 (48.4 MB/s) - ‘helper_functions.py’ saved [6733/6733]



In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2025-09-17 15:58:29--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.207, 142.250.101.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   162MB/s    in 1.0s    

2025-09-17 15:58:30 (162 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [6]:
from helper_functions import unzip_data,walk_through_dir,plot_loss_curves,create_tensorboard_callback

In [5]:
unzip_data("10_food_classes_10_percent.zip")

In [7]:
walk_through_dir('10_food_classes_10_percent')

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/tra

In [8]:
# Create  train and test directory
train_dir='10_food_classes_10_percent/train'
test_dir='10_food_classes_10_percent/test'

In [9]:
import tensorflow as tf

In [10]:
IMG_SIZE=(224,224)
BATCH_SIZE=32

In [11]:
# Instead of using ImageDataGenerator we use the below method
train_data_10_percent=tf.keras.preprocessing.image_dataset_from_directory(train_dir,label_mode='categorical',batch_size=BATCH_SIZE,image_size=IMG_SIZE)
test_data=tf.keras.preprocessing.image_dataset_from_directory(test_dir,batch_size=BATCH_SIZE,image_size=IMG_SIZE,label_mode='categorical')

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [12]:
# Check out class names
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [13]:
# See an example of batch of data
for images,label in train_data_10_percent.take(1):
  print(images,label)

tf.Tensor(
[[[[1.17285713e+02 1.95153065e+01 1.58265314e+01]
   [1.19000000e+02 2.13316326e+01 1.73367348e+01]
   [1.14076530e+02 1.95714302e+01 1.25816326e+01]
   ...
   [1.28780533e+02 1.31566269e+02 1.10352005e+02]
   [1.29933685e+02 1.32933685e+02 1.13933685e+02]
   [1.29765244e+02 1.32765244e+02 1.15765244e+02]]

  [[1.17668373e+02 2.06683674e+01 1.38571434e+01]
   [1.17000000e+02 2.29336739e+01 1.49234695e+01]
   [1.15056122e+02 2.10561218e+01 1.10561228e+01]
   ...
   [1.32173462e+02 1.35173462e+02 1.14173470e+02]
   [1.32500000e+02 1.35500000e+02 1.16500000e+02]
   [1.30780563e+02 1.33780563e+02 1.16780571e+02]]

  [[1.16428574e+02 2.17857132e+01 1.04285717e+01]
   [1.16214287e+02 2.22142868e+01 1.06428576e+01]
   [1.15785713e+02 2.17857132e+01 1.02142859e+01]
   ...
   [1.40714233e+02 1.43714233e+02 1.22714241e+02]
   [1.41525543e+02 1.44525543e+02 1.25525543e+02]
   [1.44469406e+02 1.47469406e+02 1.30469406e+02]]

  ...

  [[3.37142868e+01 1.67142849e+01 6.71428585e+00]
   [3